In [ ]:
# !pip install plotly panel

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn
from pandas import DataFrame

base_path = "."

In [ ]:
base_path = "/mnt/f/Progetti/local-llms-analyse"

### Read transaction data with categories

In [ ]:
# Read transactions_2022_2023_categorized.csv
categorized_transactions_data: DataFrame = pd.read_csv(base_path + '/transactions_2022_2023_categorized.csv')
# Add year and month columns
categorized_transactions_data["Year"] = pd.to_datetime(categorized_transactions_data["Date"]).dt.year
categorized_transactions_data["Month"] = pd.to_datetime(categorized_transactions_data["Date"]).dt.month
categorized_transactions_data["Month Name"] = pd.to_datetime(categorized_transactions_data["Date"]).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
categorized_transactions_data

In [ ]:
# For Income rows, assign Name / Description to Category
categorized_transactions_data["Category"] = np.where(
    categorized_transactions_data["Expense/Income"] == "Income",
    categorized_transactions_data["Name / Description"],
    categorized_transactions_data["Category"]
)
categorized_transactions_data

### Make pie charts - Income/ Expense breakdown

In [ ]:
def make_pie_chart(data_frame, year, label):
    # Filter the dataset for expense transactions
    sub_df = data_frame[(data_frame["Expense/Income"] == label) & (data_frame["Year"] == year)]

    color_scale = px.colors.qualitative.Set2

    pie_fig = px.pie(sub_df, values="Amount (EUR)", names="Category", color_discrete_sequence=color_scale)
    pie_fig.update_traces(textposition="inside", direction="clockwise", hole=0.3, textinfo="label+percent")

    total_expense = data_frame[(data_frame["Expense/Income"] == "Expense") & (data_frame["Year"] == year)][
        "Amount (EUR)"].sum()
    total_income = data_frame[(data_frame["Expense/Income"] == "Income") & (data_frame["Year"] == year)][
        "Amount (EUR)"].sum()

    if label == "Expense":
        total_text = f"€ {round(total_expense)}"

        # Saving rate:
        saving_rate = round((total_income - total_expense) / total_income * 100)
        saving_rate_text = f": Saving rate {saving_rate}%"
    else:
        saving_rate_text = ""
        total_text = f"€ {round(total_income)}"

    pie_fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode="hide",
        title=dict(text=f"{label} Breakdown {year} {saving_rate_text}"),
        # Add annotations in the center of the donut.
        annotations=[
            dict(
                text=total_text,
                # Square unit grid starting at bottom left of page
                x=0.5,
                y=0.5,
                font_size=12,
                # Hide the arrow that points to the [x,y] coordinate
                showarrow=False
            )
        ]
    )
    return pie_fig

In [ ]:
income_pie_fig_2022 = make_pie_chart(categorized_transactions_data, 2022, 'Income')
income_pie_fig_2022

### Make bar charts over months in a year

In [ ]:
color_scales = {
    "Income": px.colors.sequential.YlGn,
    "Expense": px.colors.sequential.OrRd,
}
def make_monthly_bar_chart(data_frame: DataFrame, year, label):
    data_frame = data_frame[(data_frame["Expense/Income"] == label) & (data_frame["Year"] == year)]
    total_by_month = (data_frame.groupby(["Month", "Month Name"])["Amount (EUR)"].sum()
                      .to_frame()
                      .reset_index()
                      .sort_values(by="Month")
                      .reset_index(drop=True))
    
    
    bar_fig = px.bar(total_by_month, x="Month Name", y="Amount (EUR)", text_auto=".2s", title=f"{label} per month",
                     color="Amount (EUR)", color_continuous_scale=color_scales[label])
    # bar_fig.update_traces(marker_color='lightslategrey')

    return bar_fig

In [ ]:
income_monthly_2022 = make_monthly_bar_chart(categorized_transactions_data, 2022, "Income")
income_monthly_2022

### Putting all charts together into tabs for 2022/2023

In [ ]:
# Pie charts
income_pie_fig_2022 = make_pie_chart(categorized_transactions_data, 2022, "Income")
expense_pie_fig_2022 = make_pie_chart(categorized_transactions_data, 2022, "Expense")
income_pie_fig_2023 = make_pie_chart(categorized_transactions_data, 2023, "Income")
expense_pie_fig_2023 = make_pie_chart(categorized_transactions_data, 2023, "Expense")

# Bar charts
income_monthly_2022 = make_monthly_bar_chart(categorized_transactions_data, 2022, "Income")
expense_monthly_2022 = make_monthly_bar_chart(categorized_transactions_data, 2022, "Expense")
income_monthly_2023 = make_monthly_bar_chart(categorized_transactions_data, 2023, "Income")
expense_monthly_2023 = make_monthly_bar_chart(categorized_transactions_data, 2023, "Expense")

# Create tabs
tabs = pn.Tabs(
    (
        "2022",
        pn.Column(
            pn.Row(
                income_pie_fig_2022,
                expense_pie_fig_2022
            ),
            pn.Row(
                income_monthly_2022,
                expense_monthly_2022
            )
        )
    ),
    (
        "2023",
        pn.Column(
            pn.Row(
                income_pie_fig_2023,
                expense_pie_fig_2023
            ),
            pn.Row(
                income_monthly_2023,
                expense_monthly_2023
            )
        )
    )
)
tabs.show()

### Create dashboard

In [ ]:
# Dashboard template
template = pn.template.FastListTemplate(
    title="Personal Finance Dashboard",
    sidebar=[
        pn.pane.Markdown("# Income Expense analysis"),
        pn.pane.Markdown(
            "Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
        pn.pane.PNG("picture.png", sizing_mode="scale_both")
    ],
    main=[
        pn.Row(
            pn.Column(
                pn.Row(tabs)
            )
        ),
    ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()